In [1]:
#default_exp showdoc

# JSX Representations Of Objects
> Render JSX representations of python classes and functions interactively.

This module is modeled after [nbdev's](https://github.com/fastai/nbdev) showdoc functionality, but is instead [Numpy Docstring](https://numpydoc.readthedocs.io/en/latest/format.html) compliant.

In [28]:
#export
from numpydoc.docscrape import NumpyDocString,ClassDoc, FunctionDoc, Parameter
from fastcore.all import test_eq, get_source_link, test
from xml.etree import ElementTree as et
import inspect, functools

In [32]:
#hide
import test_lib.example as ex

In [4]:
#export
_ATTRS_PARAMS=['Parameters', 'Attributes', 'Returns', 'Yields', 'Raises'] # These have parameters
_ATTRS_STR_LIST=['Summary', 'Extended Summary'] # These are lists of strings
def _is_func(obj): return inspect.isfunction(obj)

In [5]:
#export
def is_valid_xml(xml:str):
    "Determine if xml is valid or not."
    try: et.fromstring(xml)
    except et.ParseError as e: 
        print(f"WARNING: xml not does not parse:{e}")
        return False
    return True

You can use `is_valid_xml` to determine if JSX is valid:

In [6]:
_valid = "<Foo></Foo>" # valid jsx
assert is_valid_xml(_valid)

If you pass invalid JSX to `is_valid_xml`, a warning will be printed:

In [7]:
_invalid1 = "<Foo><Foo>"
_invalid2 = "<Foo></Bar>"

assert not is_valid_xml(_invalid1)
assert not is_valid_xml(_invalid2)

In [8]:
#export
def param2JSX(p:Parameter):
    "Format a numpydoc.docscrape.Parameters as JSX components"
    prefix = "<Parameter"
    suffix = " />"
    for a in ['name', 'type', 'desc']:
        val = getattr(p, a)
        if val:
            if a == 'desc': 
                desc = '\n'.join(val).encode('unicode_escape').decode('utf-8')
                prefix += f' {a}="{desc}"'
            else: prefix += f' {a}="{val}"'
    return prefix.strip() + suffix

In [30]:
_fd = FunctionDoc(ex.function_with_types_in_docstring)
_p = _fd['Parameters'][0]

test_eq(param2JSX(_p), '<Parameter name="param1" type="int" desc="The first parameter. something something\\nsecond line. foo" />')
assert is_valid_xml(param2JSX(_p))

In [10]:
#export
def np2jsx(obj):
    "Turn Numpy Docstrings Into JSX components"
    if inspect.isclass(obj): doc = ClassDoc(obj)
    elif _is_func(obj): doc = FunctionDoc(obj)
    else: raise ValueError(f'You can only generate parameters for classes and functions but got: {type(obj)}')
    desc_list = []
    for a in _ATTRS_STR_LIST:
        nm = a.replace(' ', '_').lower()
        desc = '\n'.join(doc[a]).encode('unicode_escape').decode('utf-8')
        if doc[a]: desc_list.append(f' {nm}="{desc}"')
    desc_props = ''.join(desc_list)
    desc_component = f'<Description{desc_props} />' if desc_props else ''
    
    jsx_sections = []
    for a in _ATTRS_PARAMS:
        params = doc[a]
        if params:
            jsx_params = '\t' + '\n\t'.join([param2JSX(p) for p in params])
            jsx_block = f'<ParamSection name="{a}">\n{jsx_params}\n</ParamSection>'
            jsx_sections.append(jsx_block)
    
    return desc_component+ '\n' + '\n'.join(jsx_sections)

Below are some examples of docstrings and resulting JSX that comes out of them. This one is of a class:

In [11]:
print(inspect.getdoc(ex.ExampleClass))

The summary line for a class docstring should fit on one line.

If the class has public attributes, they may be documented here
in an ``Attributes`` section and follow the same formatting as a
function's ``Args`` section. Alternatively, attributes may be documented
inline with the attribute's declaration (see __init__ method below).

Properties created with the ``@property`` decorator should be documented
in the property's getter method.

Attributes
----------
attr1 : str
    Description of `attr1`.
attr2 : :obj:`int`, optional
    Description of `attr2`.


In [12]:
_res = np2jsx(ex.ExampleClass)
assert '<Parameter name="attr1" type="str" desc="Description of `attr1`." />' in _res
assert 'extended_summary="If the class has public attributes' in _res
assert '</ParamSection>' in _res
print(_res)

<Description summary="The summary line for a class docstring should fit on one line." extended_summary="If the class has public attributes, they may be documented here\nin an ``Attributes`` section and follow the same formatting as a\nfunction's ``Args`` section. Alternatively, attributes may be documented\ninline with the attribute's declaration (see __init__ method below).\n\nProperties created with the ``@property`` decorator should be documented\nin the property's getter method." />
<ParamSection name="Attributes">
	<Parameter name="attr1" type="str" desc="Description of `attr1`." />
	<Parameter name="attr2" type=":obj:`int`, optional" desc="Description of `attr2`." />
</ParamSection>


This next one is of a top-level function:

In [13]:
print(inspect.getdoc(ex.function_with_types_in_docstring))

Example function with types documented in the docstring.

`PEP 484`_ type annotations are supported. If attribute, parameter, and
return types are annotated according to `PEP 484`_, they do not need to be
included in the docstring:

Parameters
----------
param1 : int
    The first parameter. something something
    second line. foo
param2 : str
    The second parameter.

Returns
-------
bool
    True if successful, False otherwise.


In [14]:
_res = np2jsx(ex.function_with_types_in_docstring)
assert 'extended_summary="`PEP 484`_ type annotations are supported' in _res
assert '<Parameter name="param2" type="str" desc="The second parameter." />' in _res
assert '<ParamSection name="Returns">' in _res
print(_res)

<Description summary="Example function with types documented in the docstring." extended_summary="`PEP 484`_ type annotations are supported. If attribute, parameter, and\nreturn types are annotated according to `PEP 484`_, they do not need to be\nincluded in the docstring:" />
<ParamSection name="Parameters">
	<Parameter name="param1" type="int" desc="The first parameter. something something\nsecond line. foo" />
	<Parameter name="param2" type="str" desc="The second parameter." />
</ParamSection>
<ParamSection name="Returns">
	<Parameter type="bool" desc="True if successful, False otherwise." />
</ParamSection>


In [15]:
#export
def fmt_sig_param(p:inspect.Parameter):
    "Format inspect.Parameters as JSX components"
    name = str(p) if str(p).startswith('*') else p.name
    prefix = f'<SigArg name="{name}" '
    if p.annotation != inspect._empty:
        prefix += f'type="{p.annotation.__name__}" '
    if p.default != inspect._empty:
        prefix += f'default="{p.default}" '
    return prefix + "/>"

`fmt_sig_param` converts individual parameters in signatures to JSX.  Let's take the complex signature below, for example:

In [16]:
_sig = inspect.signature(ex.Bar)
_sig

<Signature (a: int, b: str = 'foo', c: float = 0.1, *args, **tags)>

Each of these parameters are then converted to JSX components

In [17]:
_ps = _sig.parameters
test_eq(fmt_sig_param(_ps['a']), '<SigArg name="a" type="int" />')
test_eq(fmt_sig_param(_ps['b']), '<SigArg name="b" type="str" default="foo" />')
test_eq(fmt_sig_param(_ps['args']), '<SigArg name="*args" />')
test_eq(fmt_sig_param(_ps['tags']), '<SigArg name="**tags" />')
assert is_valid_xml(fmt_sig_param(_ps['b']))

In [18]:
#export
def get_sig_section(obj):
    "Get JSX section from the signature of a class or function consisting of all of the argument."
    if not inspect.isclass(obj) and not _is_func(obj):
        raise ValueError(f'You can only generate parameters for classes and functions but got: {type(obj)}')
    params = inspect.signature(obj).parameters.items()
    jsx_params = [fmt_sig_param(p) for _, p in params]
    return "<SigArgSection>\n" + ''.join(jsx_params) +"\n</SigArgSection>"

Let's take the class Bar, for example:

In [19]:
inspect.signature(ex.Bar)

<Signature (a: int, b: str = 'foo', c: float = 0.1, *args, **tags)>

The signature will get converted to JSX components, like so:

In [20]:
_ex_result="""<SigArgSection>
<SigArg name="a" type="int" /><SigArg name="b" type="str" default="foo" /><SigArg name="c" type="float" default="0.1" /><SigArg name="*args" /><SigArg name="**tags" />
</SigArgSection>
""".strip()

_gen_result = get_sig_section(ex.Bar)
assert is_valid_xml(_gen_result) # make sure its valid xml
test_eq(_gen_result, _ex_result)
print(_gen_result)

<SigArgSection>
<SigArg name="a" type="int" /><SigArg name="b" type="str" default="foo" /><SigArg name="c" type="float" default="0.1" /><SigArg name="*args" /><SigArg name="**tags" />
</SigArgSection>


In [21]:
#export
def get_jsx_doc(obj):
    "Construct the full JSX documentation for a particular object."
    if _is_func(obj):
        if 'self' in inspect.signature(ex.ExampleClass.example_method).parameters:
            typ = 'method'
        else: typ = 'function'
    elif inspect.isclass(obj): 
        typ = 'class'
    else: 
        raise ValueError(f'Can only parse a class or a function, but got a {type(obj)}')
    npdocs = np2jsx(obj)
    nm = f'<DocSection type="{typ}" name="{obj.__name__}" module="{inspect.getmodule(obj).__name__}" link="{get_source_link(obj)}">'
    sp = get_sig_section(obj)
    return f'{nm}\n{sp}\n{npdocs}\n</DocSection>'

`get_jsx_doc` extracts JSX Markup about an object so that you can use it for code documentation.  Here are some examples:

In [22]:
_result = get_jsx_doc(ex.ExampleClass)
assert is_valid_xml(_result)
print(_result)

<DocSection type="class" name="ExampleClass" module="test_lib.example" link="test_lib/example.py#L195">
<SigArgSection>
<SigArg name="param1" /><SigArg name="param2" /><SigArg name="param3" />
</SigArgSection>
<Description summary="The summary line for a class docstring should fit on one line." extended_summary="If the class has public attributes, they may be documented here\nin an ``Attributes`` section and follow the same formatting as a\nfunction's ``Args`` section. Alternatively, attributes may be documented\ninline with the attribute's declaration (see __init__ method below).\n\nProperties created with the ``@property`` decorator should be documented\nin the property's getter method." />
<ParamSection name="Attributes">
	<Parameter name="attr1" type="str" desc="Description of `attr1`." />
	<Parameter name="attr2" type=":obj:`int`, optional" desc="Description of `attr2`." />
</ParamSection>
</DocSection>


In [23]:
_result = get_jsx_doc(ex.Foo)
assert is_valid_xml(_result)
print(_result)

<DocSection type="class" name="Foo" module="test_lib.example" link="test_lib/example.py#L188">
<SigArgSection>
<SigArg name="a" type="int" /><SigArg name="b" type="str" default="foo" /><SigArg name="c" type="float" default="0.1" /><SigArg name="*args" /><SigArg name="**tags" />
</SigArgSection>


</DocSection>


In [34]:
#hide
### Show Documentation With `showdoc`
# todo